<a href="https://colab.research.google.com/github/anvisharma1/ObjectDetectionModel/blob/main/Object_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install mediapipe-model-maker

In [15]:
from google.colab import files
import os
import json
import tensorflow as tf

from mediapipe_model_maker import object_detector

In [16]:
import tensorflow as tf
print(tf.__version__)


2.15.1


In [43]:
train_dataset_path = "/content/drive/MyDrive/AI-Work/training"
validation_dataset_path = "/content/drive/MyDrive/AI-Work/validation"
cache_train_dir = "/content/drive/MyDrive/AI-Work/training/tmp"
cache_validation_dir = "/content/drive/MyDrive/AI-Work/validation/tmp"

In [31]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

5: AirPod Pros
1: Brown Sunglasses
2: Dark Blue Baseball Hat
3: White and pink Nike shoes
4: Yellow Pikachu Lego
0: background


In [32]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir=cache_train_dir)
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir=cache_validation_dir)
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

train_data size:  153
validation_data size:  14


In [46]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [49]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

Using existing files at /tmp/model_maker/object_detector/mobilenetmultiavg
Model: "retina_net_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 32   3704416   
                             ),                                  
                              '3': (None, 32, 32, 64             
                             ),                                  
                              '4': (None, 16, 16, 16             
                             0),                                 
                              '5': (None, 8, 8, 192)             
                             , '6': (None, 1, 1, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   144928    
                             , '4': (No

19/19 [==============================] - 148s 5s/step - total_loss: 6.2664 - cls_loss: 5.9561 - box_loss: 0.0049 - model_loss: 6.2036 - val_total_loss: 1.4553 - val_cls_loss: 1.2077 - val_box_loss: 0.0037 - val_model_loss: 1.3924
Epoch 2/30
19/19 [==============================] - 100s 5s/step - total_loss: 1.4776 - cls_loss: 1.2140 - box_loss: 0.0040 - model_loss: 1.4147 - val_total_loss: 1.3849 - val_cls_loss: 1.1469 - val_box_loss: 0.0035 - val_model_loss: 1.3221
Epoch 3/30
19/19 [==============================] - 112s 6s/step - total_loss: 1.2358 - cls_loss: 1.0676 - box_loss: 0.0021 - model_loss: 1.1730 - val_total_loss: 1.2952 - val_cls_loss: 1.0541 - val_box_loss: 0.0036 - val_model_loss: 1.2324
Epoch 4/30
19/19 [==============================] - 105s 5s/step - total_loss: 0.9775 - cls_loss: 0.8538 - box_loss: 0.0012 - model_loss: 0.9146 - val_total_loss: 1.1813 - val_cls_loss: 0.9394 - val_box_loss: 0.0036 - val_model_loss: 1.1185
Epoch 5/30
19/19 [=============================

In [50]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

4/4 [==============================] - 5s 844ms/step - total_loss: 0.7529 - cls_loss: 0.5693 - box_loss: 0.0024 - model_loss: 0.6901
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.14s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.474
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.328
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.599
 Average Recall     (AR) @[ 

In [51]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


float_ckpt.data-00000-of-00001	float_ckpt.index  metadata.json  model.tflite


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:

if category_item['id'] == 1:
  print("Brown Sunglasses")
if category_item['id'] == 2:
  print("Dark Blue Baseball Hat")
if category_item['id'] == 3:
  print(" White and pink Nike shoes")
if category_item['id'] == 4:
  print("Yellow Pikachu Lego")
if category_item['id'] == 5:
  print("AirPod Pros")
